### section 3.3: Attending to different parts of the input with self-attention


Our goal is to build a *context vector* which can be thought of as an enriched embedding vector. We take a token and get the similarity to all other tokens in the batch to produce a new embedding that gets added back, I believe.

In [1]:
import torch

In [2]:
example_text = "Your journey starts with one step"

In [3]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
     [0.55, 0.87, 0.66], # journey
     [0.57, 0.85, 0.64], # starts
     [0.22, 0.58, 0.33], # with
     [0.77, 0.25, 0.10], # one
     [0.05, 0.80, 0.55]] # step
)

Focusing on the second term - "journey"

In [4]:
query = inputs[1]

In [5]:
attn_scores_2  = torch.empty(inputs.shape[0])
for i, x in enumerate(inputs):
    attn_scores_2[i] = torch.dot(query, x)

After we generate the dot products, we want to normalize the scores. This has 2 benefits:

1. The weights all sum to one.
2. It makes for more stable training - if these values get small and we keep multiplying them with downstream weights, we can look precision.

In [6]:
attn_scores_2 / attn_scores_2.sum()

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [7]:
# verify these sum to 1
(attn_scores_2 / attn_scores_2.sum()).sum()

tensor(1.0000)

This is a more naive way to normalize - in practice people tend to use [softmax](https://www.youtube.com/watch?v=KpKog-L9veg). Which we can think of as a differentiable max function, which is helpful when we backpropagate. Let's calculate it ourselves first.

Another benefit of this is that the weights end up being positive so we can view them as probabilities of sort.

In [8]:
def naive_softmax(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [9]:
naive_softmax(attn_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

Even so, this naive calculation can run into numeric issues with large or small inputs, so it's advisable to always use the torch implementation, which has been optimized for performance. In this scenario they match!

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(attn_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [11]:
context_vector_2 = torch.zeros(query.shape)

for i, x in enumerate(inputs):
    context_vector_2 += x * attn_weights_2[i]

print(context_vector_2)


tensor([0.4419, 0.6515, 0.5683])


Now to do this for all weights. The below are not normalizaed.

In [12]:
attn_scores = torch.empty(6,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


For loops are slow. We can achieve the same output using matrix multiplication.

In [13]:
attn_scores = inputs @ inputs.T

attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [14]:
# normalizing

attn_weights = torch.softmax(attn_scores, dim=1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [15]:
# context vectors. Note we calculated row 2 by itself above

context_vectors = attn_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

### section 3.4: Implementing self-attention with trainable weights

Also known as scaled-dot-product attention. This is critical to the transformer architecture. Building off of the simple attention we completed in section 3.3, we want to generated context vectors which are weighted sums of the inputs with respect to each individual input token.

By making the weights trainable, we give the attention mechanism information to make "better" context vectors

In [16]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

Normally the dimensions in and out are the same, but here we are using different sizes to better illustrate the work.

In [17]:
# intialize the query, key, and value matrices
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

`requires_grad` is set to `False` to limit clutter in the following outputs. They need to be `True` once we start actually training.

In [18]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [19]:
keys = inputs @ W_key
values = inputs @ W_value
print(f"keys shape: {keys.shape}")
print(f"values shape: {values.shape}")

keys shape: torch.Size([6, 2])
values shape: torch.Size([6, 2])


In [20]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
attn_score_22

tensor(1.8524)

In [21]:
attn_score_2 = query_2 @ keys.T
attn_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

This scaling of the attention scores help us avoid large values going into the softmax function and producing small gradients. This can lead to slow or stagnated training.

In [22]:
# normalizing

d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_score_2 / (d_k ** 0.5), dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [23]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

So *really* what we have are 3 weight matrices that are transformations of our input tokens and they act like certain parts of a database.

* query - is the thing we're looking at. Whatever token we're focused on in a given moment.
* key - is the main lookup kind of value when our query is applied.
* value - is what that key retrieves

So the "DB" representation still needs to be learned. We transform a given token to some representation and then weigh the values by their attention score (which is a dot product of the query and keys).

### section 3.4.2: Implementing a compact self-attention Python class

We can do all of the above steps succinctly using a Python class. 

In [24]:
from utils import SelfAttention_v1

In [25]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


It makes more sense to use the built in Linear layers - they have a more sophisticated intialization scheme. 

In [26]:
from utils import SelfAttention_v2
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


### Exercise 3.1

Making sure that the 2 implementations otherwise behave the same, we will apply the weights of one to the other.

In [27]:
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)


In [28]:
sa_v1(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

### section 3.5: Hiding Future Words with Causal Attention

So it's all fine and dandy to build attention with every other token in a sequence, but it's not helpful when what we're trying to build is a model that can generate the next token. So here we will add in a mechanism that avoids the future tokens.

So we can go about building attention scores that only exist below the diagonal. First we make the typical attention weight matrix.

In [29]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / (keys.shape[-1] ** 0.5), dim=-1)
attn_weights

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [30]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple 

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

We can then multiply the attention weights by the mask. Using the classical `*` operator does an element-wise multiplication.

In [31]:
attn_weights_masked = attn_weights * mask_simple
attn_weights_masked

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

And now we need to renormalize by dividing by the sum of each column. Here we use `keepdim=True` in order to divide the entire row by the sum of that row. Without that argument, each element of the row is divided by each sum.

In [32]:
row_sums = attn_weights_masked.sum(dim=-1, keepdim=True)

In [33]:
attn_weights_normed = attn_weights_masked / row_sums
attn_weights_normed

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)

#### Information Leakage

It seems here that there is some information leakage here. We normalize using softmax with all of the values, *then* apply the mask, then renormalize. Apparently this works out mathematically. Let's test it out.

In [34]:
for i in range(0, 6):
    print(torch.softmax(attn_scores[i, :i+1]/(keys[i, :i+1].shape[0]**0.5), dim=-1))

tensor([1.], grad_fn=<SoftmaxBackward0>)
tensor([0.5517, 0.4483], grad_fn=<SoftmaxBackward0>)
tensor([0.3800, 0.3097, 0.3103], grad_fn=<SoftmaxBackward0>)
tensor([0.2758, 0.2460, 0.2462, 0.2319], grad_fn=<SoftmaxBackward0>)
tensor([0.2175, 0.1983, 0.1984, 0.1888, 0.1971], grad_fn=<SoftmaxBackward0>)
tensor([0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529],
       grad_fn=<SoftmaxBackward0>)


These match output from the masked / renormalized steps! Also checked it out mathematically. Say we have n tokens with attention scores $x_i$, where $i \in {1, 2, 3, ..., n-1, n}$. Then the softmax-ed scores are

$$
\Large\frac{e^{x_1}}{T}, \frac{e^{x_2}}{T}, \frac{e^{x_3}}{T}, ..., \frac{e^{x_{n-1}}}{T}, \frac{e^{x_n}}{T}
$$

where $\large T = \sum_{i=1}^n e^{x_i}$

Looking at the first m tokens, where $m < n$, we would mask $x_{m+1}, x_{m+2}, ..., x_n$ and divide by the sum of the remaining elements. So,

$$
\LARGE\frac{\frac{e^{x_1}}{T}}{\frac{\sum_{i=1}^m e^{x_i}}{T}}, \frac{\frac{e^{x_2}}{T}}{\frac{\sum_{i=1}^m e^{x_i}}{T}}, ..., \frac{\frac{e^{x_m}}{T}}{\frac{\sum_{i=1}^m e^{x_i}}{T}} 
$$

Simplifying to

$$
\Large\frac{e^{x_1}}{\sum_{i=1}^m e^{x_i}}, \frac{e^{x_2}}{\sum_{i=1}^m e^{x_i}}, ..., \frac{e^{x_m}}{\sum_{i=1}^m e^{x_i}}
$$

.|..

Note this sums to 1!

#### Now we can get even cuter

We can cut down on steps by instead masking the elements above the diagonal with negative infinity values and then using softmax. Those items will evaluate to 0.


In [35]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
masked

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)

In [36]:
attn_weights = torch.softmax(masked / (keys.shape[-1] ** 0.5), dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

### section 3.5.2: Masking additional attention weights with dropout

we could use the above attention weights to generate our context vectors, but we can add one more layer to help with overfitting - DROPOUT.

In [37]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6, 6)
dropout(example)

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])

So even though we started with a matrix of 1s, when we apply drop out the remaining active elements get scaled by by a value equal to the dropout rate. So 1/0.5 = 2. This helps keep the matrix stable.

In [38]:
dropout(attn_weights)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4925, 0.4638, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3941, 0.0000],
        [0.3869, 0.3327, 0.0000, 0.3084, 0.3331, 0.3058]],
       grad_fn=<MulBackward0>)

### section 3.5.3: Implementing a compact causal attention class

We will incorporate these new concepts into our SelfAttention class we've already made. We'll also want to make sure that the class can handle multiple batches. 

In [39]:
batch = torch.stack((inputs, inputs), dim =0)

In [40]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [41]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

register_buffer is a way to capture some tensors that are important to the model, but not trained - like our masking algorithm.

In [42]:
from utils import CausalAttention

In [43]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([2, 6, 2])


### section 3.6: Extending single-head attention to multi-head attention

To go from single to multi headed, we just have multiple single heads running in parallel. We implement them with their own weights and then consolidate them in some way.

The first naive way to do this is to just stack our attention classes in a wrapper class.

In [44]:
from utils import MultiHeadAttentionWrapperClass

torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapperClass(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


#### Exercise 3.2: Returning 2-dimensional embedding vectors 

How do we output 2 dimensional contexts while staying with the 2 heads? Just change the d_out to 1 - each CausalAttention class will output 1 dimension and then those would be concatenated.

In [45]:
context_length = batch.shape[1]
d_in, d_out = 3, 1

mha = MultiHeadAttentionWrapperClass(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.0189, 0.2729],
         [0.2181, 0.3037],
         [0.2804, 0.3125],
         [0.2830, 0.2793],
         [0.2476, 0.2541],
         [0.2748, 0.2513]],

        [[0.0189, 0.2729],
         [0.2181, 0.3037],
         [0.2804, 0.3125],
         [0.2830, 0.2793],
         [0.2476, 0.2541],
         [0.2748, 0.2513]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


### 3.6.2 Implementing Multihead Attention with weight splits

How we built this before is inefficient (sequential). We can parallelize this using matrix multiplication.